# Turkey

In [3]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

In [ ]:
params = Params(
    r_outer = ,
    r_inner = ,
    T_oven = ,
    )

In [19]:
def make_system(params, t_end):
    """Make a system object for the turkey model.
    inner is inner temperature of turkey
    outer is outer temperature of turkey
    
    params is the r and C values for thermal compartments
    t_end is when the turkey is taken out of the oven
    )
    
    returns: System object
    """
    #initial temperatures are of the pre-heated oven
    init = State(T_inner=T_oven, T_outer=T_oven)

    return System(init=init, t0=0, t_end=t_end,
                  params = params)

#state(outer, inner)
#system(init, t0, t_end, parameters)

In [12]:
def slope_func(state, t, system):
    """Makes the differential equations.
    
    state: State (t_inner, t_outer)
    t: time
    system: System object
    
    returns: didt, dodt
    """
    
    unpack(system)
    # ... = params
    T_inner, T_outer = state
    
    
    dodt = r_outer * (T_oven - T_outer)
    
    #compute inner temperature change
    didt = r_inner * (T_inner - T_outer)

    return didt, dodt

In [9]:
def plot_results(inner, outer):
    """Plot the inner and outer temperatures of the Turkey
    
    inner: TimeSeries
    outer: TimeSeries
    
    """
    plot(t_inner, '-', label='Inner Turkey Temp')
    plot(t_outer, '-', label='Outer TurkeyTemp')
    
    decorate(xlabel='Minutes',
             ylabel='Temperature')

http://www.greenbuildingadvisor.com/sites/default/files/Modeling%20Heat%20Transfer%20of%20Turkeys.pdf data from turkey
 118°C oven temp, with the oven at room temperature starting their experiment
 thermal conductivity 0.461 W/m K
 surface heat transfer coefficient of 19.252 W/m2
K